In [1]:
import polars as pl
from op_datasets.processing.execute import reader, BatchInput
from op_datasets.logic.transforms.daily_address_summary import (
    daily_address_summary,
    CONDITIONS,
)
from op_coreutils.logger import structlog
from op_coreutils.bigquery.write import overwrite_partitions

from datetime import datetime
import os

os.environ["OPLABS_ENV"] = "PROD"


log = structlog.get_logger()

2024-10-17 13:36:35 [info     ] Loaded env var: OP_ANALYTICS_VAULT


In [2]:
inputbatch: BatchInput = None
for _inputbatch in reader(
    chain="op", block_spec="126071012:+500", source_spec="goldsky"
):
    inputbatch = _inputbatch
    break

dataframes = inputbatch.dataframes

2024-10-17 13:36:36 [info     ] Loaded 5 items into vault.    
2024-10-17 13:36:37 [info     ] Initialized Clickhouse client.
2024-10-17 13:36:38 [info     ] Query success: blocks in 2.43s 2000 rows
2024-10-17 13:36:39 [info     ] Query success: transactions in 3.72s 43972 rows
2024-10-17 13:36:39 [info     ] Query success: logs in 3.78s 187138 rows


In [3]:
dataframes

{'blocks': shape: (1_012, 24)
 ┌───────┬─────────┬──────────┬────────────┬───┬──────────┬───────────┬────────────┬────────────────┐
 │ chain ┆ network ┆ chain_id ┆ dt         ┆ … ┆ gas_used ┆ gas_limit ┆ extra_data ┆ transaction_co │
 │ ---   ┆ ---     ┆ ---      ┆ ---        ┆   ┆ ---      ┆ ---       ┆ ---        ┆ unt            │
 │ str   ┆ str     ┆ i32      ┆ str        ┆   ┆ i64      ┆ i64       ┆ str        ┆ ---            │
 │       ┆         ┆          ┆            ┆   ┆          ┆           ┆            ┆ i64            │
 ╞═══════╪═════════╪══════════╪════════════╪═══╪══════════╪═══════════╪════════════╪════════════════╡
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 1779886  ┆ 30000000  ┆ 0x         ┆ 14             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 4285527  ┆ 30000000  ┆ 0x         ┆ 27             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 6187833  ┆ 30000000  ┆ 0x         ┆ 30             │
 │ op    ┆ mainnet ┆ 10       ┆ 2024-09-30 ┆ … ┆ 516

In [4]:
# check if our blocks are in the range we want

log.info(f"Min block number: {dataframes['blocks']['number'].min()}")
log.info(f"Max block number: {dataframes['blocks']['number'].max()}")
log.info(
    f"Min block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].min())}"
)
log.info(
    f"Max block number: {datetime.fromtimestamp(dataframes['blocks']['timestamp'].max())}"
)

2024-10-17 13:36:39 [info     ] Min block number: 126070000   
2024-10-17 13:36:39 [info     ] Max block number: 126071011   
2024-10-17 13:36:39 [info     ] Min block number: 2024-09-30 19:26:17
2024-10-17 13:36:39 [info     ] Max block number: 2024-09-30 19:59:59


In [5]:
result = daily_address_summary(
    dataframes["transactions"],
    conditions=CONDITIONS,
    address="from_address",
    chain_id="chain_id",
    chain="chain",
    dt="dt",
)

In [6]:
result.head()

from_address,chain_id,chain,dt,total_txs,total_txs_success,total_txs_fail,total_blocks,total_blocks_success,total_blocks_fail,min_block_number,max_block_number,block_interval_active,min_nonce,max_nonce,nonce_interval_active,total_l2_gas_used,total_l2_gas_used_success,total_l2_gas_used_fail,total_l1_gas_used,total_l1_gas_used_success,total_l1_gas_used_fail,total_gas_fees,total_gas_fees_success,total_gas_fees_fail,l2_contrib_gas_fees,l1_contrib_gas_fees,l1_blobgas_contrib_gas_fees,l1_l1gas_contrib_gas_fees,l2_contrib_gas_fees_base_fee,l2_contrib_gas_fees_priority_fee,avg_l2_gas_price_gwei,avg_l2_base_fee_gwei,avg_l2_priority_fee_gwei,avg_l1_gas_price_gwei,avg_l1_blob_base_fee_gwei,min_block_timestamp,max_block_timestamp,time_interval_active,unique_hours_active,perc_daily_hours_active,num_to_addresses,num_to_addresses_success,num_to_addresses_fail
str,i32,str,str,u32,u32,u32,u32,u32,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,"decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,26]","decimal[*,26]","decimal[*,26]","decimal[*,17]","decimal[*,17]",datetime[μs],datetime[μs],duration[μs],u32,f64,u32,u32,u32
"""0xf7df48764762810210e35e6b8022…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070519,126070519,1,5,5,1,32202,32202,0,1600,1600,0,0.0000001323269943920000,0.0000001323269943920000,0.0000000000000000000000,0.0000000711252014400000,0.0000000612017929520000,0.0000000016227408000000,0.0000000000000000000000,0.0000000711252014400000,0.0000000000000000000000,0.00220872000000000000000000,0.00220872000000000000000000,0.00000000000000000000000000,7.31798747500000000,0.00000000100000000,2024-09-30 23:43:35,2024-09-30 23:43:35,0µs,1,0.041667,1,1,0
"""0x12661f2fc8d0f88594392c2b3040…",10,"""op""","""2024-09-30""",22,22,0,22,22,0,126070278,126070930,653,5024,5045,22,3740698,3740698,0,48622,48622,0,0.0001006474819778750000,0.0001006474819778750000,0.0000000000000000000000,0.0000987665453594860000,0.0000018809366183890000,0.0000000493130644860000,0.0000000000000000000000,0.0000043714262751360000,0.0000943951190843500000,null,null,null,7.39857998442273045,0.00000000100000000,2024-09-30 23:35:33,2024-09-30 23:57:17,21m 44s,1,0.041667,1,1,0
"""0x2b5eab064f8b4db23aaab03365d1…",10,"""op""","""2024-09-30""",4,4,0,4,4,0,126070079,126070987,909,14977,14980,4,285051,285051,0,15568,15568,0,0.0000011508027595500000,0.0000011508027595500000,0.0000000000000000000000,0.0000005735407861840000,0.0000005772619733660000,0.0000000157892679840000,0.0000000000000000000000,0.0000002884897861840000,0.0000002850510000000000,0.00201206375765740165794892,0.00101206375765740165794892,0.00100000000000000000000000,7.09286575696762589,0.00000000100000000,2024-09-30 23:28:55,2024-09-30 23:59:11,30m 16s,1,0.041667,1,1,0
"""0x48b2a539ae4a3fbd56e40a228cdd…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070503,126070503,1,17,17,1,36702,36702,0,1600,1600,0,0.0000001418928840180000,0.0000001418928840180000,0.0000000000000000000000,0.0000000790152586740000,0.0000000628776253440000,0.0000000016227408000000,0.0000000000000000000000,0.0000000790152586740000,0.0000000000000000000000,0.00215288700000000000000000,0.00215288700000000000000000,0.00000000000000000000000000,7.51836919400000000,0.00000000100000000,2024-09-30 23:43:03,2024-09-30 23:43:03,0µs,1,0.041667,1,1,0
"""0x56ad2cd6ad6f52c72181b93ac66d…",10,"""op""","""2024-09-30""",1,1,0,1,1,0,126070351,126070351,1,102457,102457,1,48512,48512,0,1600,1600,0,0.0000001545823451140000,0.0000001545823451140000,0.0000000000000000000000,0.0000000989821383680000,0.0000000556002067460000,0.0000000016227408000000,0.0000000000000000000000,0.0000000504701383680000,0.0000000485120000000000,0.00204036400000000000000000,0.00104036400000000000000000,0.00100000000000000000000000,6.64819765700000000,0.00000000100000000,2024-09-30 23:37:59,2024-09-30 23:37:59,0µs,1,0.041667,1,1,0


In [7]:
overwrite_partitions(result, "temp", "daily_address_summary", expiration_days=999)

2024-10-17 13:36:39 [info     ] Writing 1 partitions to BQ [2024-09-30 00:00:00 ... 2024-09-30 00:00:00]
2024-10-17 13:36:40 [info     ] DRYRUN OVERWRITE PARTITION: Wrote 2.3Krows 289.7KB to BQ temp.daily_address_summary
